In [26]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [27]:
%pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import nltk
import string
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
 


In [6]:
nltk.download('stopwords')
#download the stopwords

real = pd.read_csv('data/True.csv')
fake = pd.read_csv('data/Fake.csv')
real.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\64pul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
fake['label'] = 0
real['label']=1

In [15]:
#combining the datasets
data = pd.concat([fake, real], ignore_index=True)

#keep only text and label columns remove the rest
data = data[['text', 'label']]

#reshuffle the dataset
data = data.sample(frac=1, random_state=42)

#preproccesing
def preprocess(text):
  text = text.lower()
  text = ''.join([c for c in text if c not in string.punctuation])
  stop_words = stopwords.words('english')
  text = ''.join([word for word in text.split() if word not in string.punctuation])
  return text


#applying the preprocessing
data['text'] = data['text'].apply(preprocess)

#spliting the dataset into testing and training data
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

In [16]:
#converting the text features to numerical features
# using tf-dif
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test) 
#all into numerical data has been converted

In [19]:
#training the logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

#evaluate model on the test set
y_pred = model.predict(X_test_tfidf)
print("ACCURACY: ", accuracy_score(y_test, y_pred))

print("classfication report: \n", classification_report(y_test, y_pred))


ACCURACY:  0.6726057906458798
classfication report: 
               precision    recall  f1-score   support

           0       0.62      1.00      0.76      4710
           1       1.00      0.31      0.47      4270

    accuracy                           0.67      8980
   macro avg       0.81      0.66      0.62      8980
weighted avg       0.80      0.67      0.63      8980



In [21]:
import pickle

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open('fake_news_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [30]:
#confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap= 'Blues')
plt.title('confusion matrix')
plt.xlabel('predicted')
plt.ylabel('actual')
plt.savefig('confusion_matrix_for_fakeNews.png')
plt.close()


In [31]:
%pip install wordCloud

  Using cached wordcloud-1.9.4-cp313-cp313-win_amd64.whl.metadata (3.5 kB)
Using cached wordcloud-1.9.4-cp313-cp313-win_amd64.whl (300 kB)
Note: you may need to restart the kernel to use updated packages.


In [32]:
from wordcloud import WordCloud

In [34]:
fakeWords = ' '.join(fake['text'].tolist())
realWords = ' '.join(real['text'].tolist())

wc_fake = WordCloud(width=800, height=400, background_color='white').generate(fakeWords)
wc_real = WordCloud(width=800, height=400, background_color='white').generate(realWords)

wc_fake.to_file('wordcloud_fake.png')
wc_real.to_file('wordcloud_real.png')